# Hugging Face evaluate

La librería `Evaluate` de `Hugging Face` es una librería para evaluar fácilmente modelos y datasets.

Con una sola línea de código, se tiene acceso a docenas de métodos de evaluación para diferentes dominios (NLP, computer vision, reinforcement learning y más). Ya sea en tu máquina local, o en una configuración de entrenamiento distribuida, puedes evaluar modelos de manera consistente y reproducible

En la página de [evaluate](https://huggingface.co/evaluate-metric) en Hugging Face se puede obtener una lista completa de las métricas disponibles. Cada métrica tiene un espacio dedicado con una demostración interactiva sobre cómo usar la métrica y una tarjeta de documentación que detalla las limitaciones y el uso de las métricas.

## Instalación

Para instalar la librería es necesario hacer 

``` bash
pip install evaluate
```

## Tipo de evaluaciones

Hay varios tipos de evaluaciones disponibles

 * `metric`: Una métrica se utiliza para evaluar el rendimiento de un modelo y, por lo general, incluye las predicciones del modelo y las etiquetas ground truth.
 * `comparison`: Se utiliza para comparar dos modelos. Esto se puede hacer, por ejemplo, comparando sus predicciones con etiquetas ground truth.
 * `measurement`: El dataset es tan importante como el modelo entrenado en él. Con las mediciones se pueden investigar las propiedades de un dataset.

## Carga

Cada `metric`, `comparison` o `measurement` se puede cargar con el método load

In [1]:
import evaluate

accuracy = evaluate.load("accuracy")
accuracy

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

Si quieres estar seguro de cargar el tipo de métrica que deseas, si tipo `metric`, `comparison` o `measurement`, puedes hacerlo añadiendo el parámetro `module_type`

In [6]:
import evaluate

accuracy = evaluate.load("accuracy", module_type="metric")
word_length = evaluate.load("word_length", module_type="measurement")

[nltk_data] Downloading package punkt to
[nltk_data]     /home/maximo.fernandez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Carga de módulos de la comunidad

A parte de los propios módulos que ofrece la librería, también puedes cargar modelos que haya subido alguién al hub de Hugging Face

In [7]:
element_count = evaluate.load("lvwerra/element_count", module_type="measurement")

### Lista de módulos disponibles

Si queremos obtener una lista de todos los módulos disponibles tenemos que usar el método `list_evaluation_modules`, en el podemos poner filtros de búsqueda

In [12]:
evaluate.list_evaluation_modules(
  module_type="comparison",
  include_community=True,
  with_details=True)

[{'name': 'ncoop57/levenshtein_distance',
  'type': 'comparison',
  'community': True,
  'likes': 0},
 {'name': 'kaleidophon/almost_stochastic_order',
  'type': 'comparison',
  'community': True,
  'likes': 1}]

## Atributos del módulo

Todos los módulos de evaluación vienen con una variedad de atributos útiles que ayudan a utilizar el módulo, estos atributos son

|Atributo|Descripción|
|---|---|
|description|Una breve descripción del módulo de evaluación.|
|citation|Una cadena BibTex para citar cuando esté disponible.|
|features|Un objeto Features que define el formato de entrada.|
|inputs_description|Esto es equivalente a la cadena de documentación de los módulos.|
|homepage|La página de inicio del módulo.|
|license|La licencia del módulo.|
|codebase_urls|Enlace al código detrás del módulo.|
|reference_urls|URL de referencia adicionales.|

Veamos algunos

In [14]:
accuracy = evaluate.load("accuracy")

In [17]:
print(f"description: {accuracy.description}")
print(f"\ncitation: {accuracy.citation}")
print(f"\nfeatures: {accuracy.features}")
print(f"\ninputs_description: {accuracy.inputs_description}")
print(f"\nhomepage: {accuracy.homepage}")
print(f"\nlicense: {accuracy.license}")
print(f"\ncodebase_urls: {accuracy.codebase_urls}")
print(f"\nreference_urls: {accuracy.reference_urls}")

description: 
Accuracy is the proportion of correct predictions among the total number of cases processed. It can be computed with:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
 Where:
TP: True positive
TN: True negative
FP: False positive
FN: False negative


citation: 
@article{scikit-learn,
  title={Scikit-learn: Machine Learning in {P}ython},
  author={Pedregosa, F. and Varoquaux, G. and Gramfort, A. and Michel, V.
         and Thirion, B. and Grisel, O. and Blondel, M. and Prettenhofer, P.
         and Weiss, R. and Dubourg, V. and Vanderplas, J. and Passos, A. and
         Cournapeau, D. and Brucher, M. and Perrot, M. and Duchesnay, E.},
  journal={Journal of Machine Learning Research},
  volume={12},
  pages={2825--2830},
  year={2011}
}


features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}

inputs_description: 
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    nor

## Ejecución

Ahora que sabemos cómo funciona el módulo de evaluación y qué debe contener, vamos a usarlo. Cuando se trata de calcular la evaluación, hay dos formas principales de hacerlo:

 * Todo en uno
 * Incremental

En el enfoque incremental, las entradas necesarias se agregan al módulo con `EvaluationModule.add()` o `EvaluationModule.add_batch()` y la puntuación se calcula al final con `EvaluationModule.compute()`. Alternativamente, se pueden pasar todas las entradas a la vez a `compute()`.

Veamos estos dos enfoques.

### Todo en uno

Una vez tenemos todas las predicciones y ground truths podemos calcular la métrica. Una vez que tenemos un módulo definido, le pasamos las predicciones y los ground truth mediante el método `compute()`

In [18]:
accuracy = evaluate.load("accuracy")

In [19]:
predictions = [1, 0, 0, 1]
targets = [0, 1, 0, 1]

accuracy_value = accuracy.compute(predictions=predictions, references=targets)
accuracy_value

{'accuracy': 0.5}

### Incremental

En muchos procesos de evaluación, las predicciones se construyen de forma iterativa, como en un bucle for. En ese caso, podrías almacenar las predicciones y ground truths en una lista y al final pasarlas a `compute()`.

Sin embargo con los métodos `add()` y `add_batch()` puedes evitar el paso de almacenar las predicciones.

Si tienes todas las predicciones de un solo batch hay que usar el método `add()`

In [20]:
for ref, pred in zip([0,1,0,1], [1,0,0,1]):
    accuracy.add(references=ref, predictions=pred)
accuracy_value = accuracy.compute()
accuracy_value

{'accuracy': 0.5}

Sin embargo, cuando se tienen predicciones de varios batches se tiene que usar el método `add_batch()`

In [21]:
for refs, preds in zip([[0,1],[0,1]], [[1,0],[0,1]]):
    accuracy.add_batch(references=refs, predictions=preds)
accuracy_value = accuracy.compute()
accuracy_value

{'accuracy': 0.5}

## Combinación de varias evaluaciones